# Computational Theory: Analysis of the Countdown Numbers Game

*Jakub Prochnicki G00373793*

---

# Table of Contents
1. [Introduction](#intro)
2. [Explanation of the Countdown Numbers Game](#concepts)
3. [Analysis of the Computational Complexity](#oracle)
4. [Brute Force Approach](#implementation)
5. [Reverse Polish Notation](#conclusion)
6. [Recursive Approach](#rpn)
7. [Conclusion](#conclusion)

# Introduction
---

This notebook constitutes my assessment submission for the Computational Theory Module at ATU Galway. The aim of this notebook is to provide an analysis of the Countdown Numbers Game. Firstly, I will provide an explanation of the game and give an example of the game in action. Following the explanation, I delve into an analysis of the computational complexity of the game and provide various approaches to solving the game, namely: Brute Force Approach, Heuristic Approach and Reverse Polish Notation. Lastly, I will evaluate my findings from solving the problem using various approaches

# Explanation of the Countdown Numbers Game
---
The Countdown Numbers Game allows people to showcase their math skills on a British aired TV Show, Countdown. The aim of the game is to reach a specific target number using a set of six other numbers and basic arithmetic operations. In order to better understand the game, I have come up with the following breakdown of how a round of Countdown works.

- The player selects six numbers from two groups : The first group consisting of four large numbers: 25, 50, 75, and 100. The other group holds twenty smaller numbers ranging from 1 to 10, with each number appearing twice. The player has the freedom to use any combination of numbers from these two groups.

- The game generates a target number, a three digit number.
- The player is then given a 30 second window and is given the task of using a combination of arithmetic operations in order to come as close as possible to the target number. Each of the six numbers can be used once and players are not obliged to use all six numbers. The operations must however result in a whole number at every step.
- The score is determined by the closeness of the player's number to the target score. A perfect score of 10 is awarded to players where they have reached the target number.

  *Example of the Game In Action*

  

# Analysing the Computational Complexity of the Game
---

*The Problem:* In the game, a player is given six numbers and must use arithmetic operations (addition, subtraction, multiplication, and division) to reach a randomly generated three-digit target number. The operations must result in whole numbers, and each of the six numbers can be used at most once.


In order to analyse the computational complexity, it is necessary to consider how difficult it is to find a solution to reach the target number using a set of numbers and arithmetic operations. This analysis typically focuses on the theoretical limits of solving this problem using algorithms, and the complexity can be described in terms of both time and space (memory). 

The primary challenge is the *combinatorial nature* of the problem. For each of the six numbers, there are decisions about:

 - Whether to use the number or not.
 - Which arithmetic operation to apply.
 - Which other number to combine it with.

As there are four operations and up to five other numbers to combine with, the possibilities increase exponentially with each step.
 
*Permutations:* The numbers can be arranged in 6 different ways, resulting in a factorial of 6

*Operations:* At each step between two numbers, you can apply one of four operations. If you consider operations between pairs of numbers, the number of operational combinations also increases rapidly.

*Branching Factor:* If we take a decision tree approach, each level of the tree would involve choosing a number (up to 6 choices), choosing another number (up to 5 remaining choices), and picking an operation (4 choices). This results in a branching factor of 6 x 5 x 4 = 120 at the first level of the decision tree alone.

*Time Complexity:* We can consider the time complexity of the problem to be exponential when considering all permutations and combinations of operations and number uses.

*Space Complexity:* The space complexity of this problem refers to the need of storing results and calculations. Like the time complexity, this grows with the number of calculations.

*NP - Completeness*: 

The author of (https://www.semanticscholar.org/reader/21f455ca0bbf9a355611bc0593dd1cf8a8d32584) pays attention to the NP completeness of arithemtic expressions. 



# Implementing Countdown using a Brute Force Approach
---

The Brute Force Approach is the first approach to solving the game that I demonstrate in this notebook. This approach was explained by the author of(https://www.daitx.com/2016/05/01/countdown-math/), where he also takes a brute force approach to the problem and pays attention to reverse polish notation. In this approach, you must generate all the  possible number and operation combinations and evaluate each combination to see if it can produce the target number. This implementation solves the problem by creating all possible permutations of operations and numbers. It then checks each permutation to see if it is equal to the target number. Using this approach ensures that a solution will be found if it exists however it comes with the cost of a high computational complexity which I confirm by using Big O notation and measuring time complexity.

An implementation of the countdown game using this approach is demonstrated using the code below which was generated by ChatGPT using the following prompt:

```
Implement the countdown numbers game in python using a brute force approach.
```


In [113]:
import itertools
import operator
import time
import threading

def eval_expression(expr):
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    stack = []
    try:
        for token in expr:
            if token in ops:
                b, a = stack.pop(), stack.pop()
                if token == '/' and b == 0:
                    return None  # Avoid division by zero
                stack.append(ops[token](a, b))
            else:
                stack.append(token)
        return stack[0] if len(stack) == 1 else None
    except Exception:
        return None

def generate_expressions(numbers, operations):
    for num_perm in itertools.permutations(numbers):
        for ops_perm in itertools.permutations(operations, len(num_perm) - 1):
            expression = []
            nums = iter(num_perm)
            expression.append(next(nums))
            for op in ops_perm:
                expression.append(op)
                expression.append(next(nums))
            yield expression

def solve_numbers(numbers, target, timeout=10):
    operations = ['+', '-', '*', '/'] * (len(numbers) - 1)
    start_time = time.time()
    for expr in generate_expressions(numbers, operations):
        if eval_expression(expr) == target:
            return expr
        if time.time() - start_time > timeout:
            return None
    return None

# Example usage
numbers = [25, 50, 3, 7, 8, 2]
target = 843
solution = solve_numbers(numbers, target, timeout=1)

if solution:
    print("Solution found:", solution)
else:
    print("None")


None


Here, we can see an example usage of the game using this approach. This approach failed to find a solution within 10 seconds, highlighting the high computational complexity of the approach.

Big O Notation, a mathematical notation used to describe the upper bound of the time complexity of an algorithm can be used to determine the computational complexity of this approach. We can do this by looking at the operations performed by the functions.

*Generation Complexity*

- ``` generate_expressions ``` : The complexity of generating these expressions comes from the permutations of operations and permutations of numbers.

1. **Permutations of Operations** : ```itertools.permutations(operations, len(numbers) - 1)``` generates permutations of the operations. If there are n numbers, there will be n-1 slots for operations. Since each slot can hold any of the four operations (+, -, *, /), the total number of permutations of operations is 4^(n-1).

2. **Permutations of Number** : For n numbers, the number of ways to order these numbers is n!, the factorial of n.

This means that the complexity of generating all possible expressions is O(n! * 4^(n-1))

*Evaluation Complexity*

- ``` eval_expressions ``` : the complexity of evaluating an expression depends largely on the length of the expression, which is proportional to 2n - 1 for n numbers (each number except the last is followed by an operation). The complexity of evaluating a single expression is O(n), as each token (number or operator) in the expression is processed once.

Each expression evaluation takes O(n), and there are n! * 4^(n-1) such evaluations.

*Total Computational Complexity*

The total computational complexity is O(n! * 4^(n-1) * n), which yields a result of 4,423,680 calculations where n is 6, the size of the number array. 





# Reverse Polish Notation
---

Reverse Polish Notation (RPN), also known as postfix notation is a mathematical notation invented in the 1950s that places the arithmetic operator after the arguments it is being operated on. In RPN there is no need for brackets to indicate the grouping of the terms, instead the expression is evaluated from left to right. This simplifies the computation of the expression which is highly favourable in the problem of solving the Countdown numbers game.(https://mathworld.wolfram.com/ReversePolishNotation.html) 

The figure below depicts the stack structure of an RPN expression, where we can see the following actions being performed: 

 - Push the next appearing number to the stack.
 - Pop the number of the stack twice where an operator appears and push the result of the  operation.

   ADDING PICTURE OF RPN STACK HERE

Using RPN in the context of the Countdown game is beneficial in reducing the computational complexity of the game as it simplifies the process of mathematical calculations

The following steps must be taken in order to implement the Countdown numbers game using Reverse Polish Notation: 

  1. **Generate valid RPN expressions**: This step involves confirming that the RPN expressions are valid by ensuring that the count of numbers is always greater than the count of operations up to that point, at every point in the sequence.
     
  2. **Evaluate RPN expressions**: In this step, an RPN stack that I explained previously is used in order to evaluate the expression. If the result of the expression matches the target number, the expression is a solution.
     
  3. **Generate Combinations**: Handle the generation of all possible number combinations.


The code below demonstrates Reverse Polish Notation by applying it to a brute force approach.


In [114]:
import itertools
import operator
import time

def generate_valid_rpn(nums, ops):
    """Generate all valid RPN sequences from numbers and a set of operations."""
    if len(nums) == 1:
        yield nums
    else:
        # Generate all valid sequences by selecting each pair of numbers to combine with an operator
        for i in range(1, len(nums)):
            for left in generate_valid_rpn(nums[:i], ops):
                for right in generate_valid_rpn(nums[i:], ops):
                    for op in ops:
                        yield left + right + [op]

def eval_rpn(expr):
    """Evaluate an expression in Reverse Polish Notation."""
    ops = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}
    stack = []
    try:
        for token in expr:
            if token in ops:
                if len(stack) < 2:
                    return None  # Not enough operands
                b, a = stack.pop(), stack.pop()
                if token == '/' and b == 0:
                    return None  # Avoid division by zero
                result = ops[token](a, b)
                stack.append(result)
            else:
                stack.append(token)
        return stack[0] if len(stack) == 1 else None
    except:
        return None

def solve_numbers_rpn(numbers, target):
    operations = ['+', '-', '*', '/']
    possible_numbers = list(itertools.permutations(numbers))
    start_time = time.time()  # Start measuring time
    for number_sequence in possible_numbers:
        for expr in generate_valid_rpn(list(number_sequence), operations):
            if eval_rpn(expr) == target:
                execution_time = time.time() - start_time  # Calculate execution time
                return expr, execution_time
    execution_time = time.time() - start_time  # Calculate execution time if no solution found
    return None, execution_time

# Example usage
numbers = [25, 50, 3, 7, 8, 2]
target = 843
solution, execution_time = solve_numbers_rpn(numbers, target)

if solution:
    print("Solution found:", solution)
    print("Execution time:", execution_time, "seconds")
else:
    print("No solution found.")
    print("Execution time:", execution_time, "seconds")



Solution found: [25, 50, 8, '+', 7, '*', 3, '+', 2, '*', '+']
Execution time: 1.1922264099121094 seconds


# Recursive Search
---

This approach was demonstrated by the author of (https://www.cgjennings.ca/articles/countdown-numbers/). This approach tries to find the target number by applying recursively selected numbers and operations. The aim is to make intelligent choices that reduce the number of possibilities to explore, minimizing unnecessary calculations. This approach adheres to the following strategy: 


- Base Case : Stop the search if the target is reached or there is no more numbers to use.
- Recursive Calls: For each number left in the pool, try all operations with the current result and recursively continue to see if you can reach the target.
- Pruning : If any operation results in a negative number or a non-integer, the branch of that recursion is pruned.
- State Management : Manage the state including the current result and remaining numbers.

An implementation of the countdown game using this approach is demonstrated using the code below which was generated by ChatGPT using the following prompt:

```
Generate an implementation of the countdown game using the recursive search approach.
```


In [115]:
import operator
import time

def countdown_recursive(numbers, current_result, target, used_numbers, operations):
    """
    Recursive function to solve the countdown game.
    
    Args:
    numbers (list): List of available numbers.
    current_result (int): Current result of the operations.
    target (int): Target number to reach.
    used_numbers (list): Numbers that have already been used.
    operations (dict): Available operations with their respective functions.
    
    Returns:
    str: String representation of the operations leading to the target, or None if not found.
    """
    # Base case: Check if the current result is the target
    if current_result == target:
        return ""
    if not numbers:  # No numbers left to try
        return None
    
    for i, num in enumerate(numbers):
        if num in used_numbers:
            continue  # Skip numbers that have been used
        for symbol, func in operations.items():
            # Apply the operation if it's valid (non-negative results, integral divisions)
            try:
                next_result = func(current_result, num)
                if symbol == '/' and current_result % num != 0:
                    continue  # Skip non-integral divisions
                if next_result < 0:
                    continue  # Skip negative results
            except ZeroDivisionError:
                continue  # Skip division by zero
            
            # Mark this number as used and recurse
            used_numbers.add(num)
            result = countdown_recursive(numbers, next_result, target, used_numbers, operations)
            if result is not None:
                return f"{current_result} {symbol} {num} = {next_result}\n{result}"
            # Backtrack: Unmark this number as used
            used_numbers.remove(num)
    
    return None

def solve_countdown(numbers, target):
    operations = {
        '+': operator.add,
        '-': operator.sub,
        '*': operator.mul,
        '/': operator.truediv
    }
    for num in numbers:
        # Try starting the calculation with each number
        used_numbers = set([num])
        start_time = time.time()
        result = countdown_recursive(numbers, num, target, used_numbers, operations)
        execution_time = time.time() - start_time
        if result is not None:
            return f"Start with {num}:\n{result.strip()}\nExecution time: {execution_time} seconds"
    return "No solution found"

# Example usage
numbers = [25, 50, 3, 7, 8, 2]
target = 845
print(solve_countdown(numbers, target))


Start with 25:
25 + 8 = 33
33 * 7 = 231
231 + 50 = 281
281 * 3 = 843
843 + 2 = 845
Execution time: 0.019740581512451172 seconds


To summarise the recursive approach. let's look at the two functions in the code above.

- ```countdown_recursive``` takes the current state and attemps to reach the target number by applying each operation to each number. The pruning strategy I outline earlier is also used here to avoid negative results and division of non-integers.

- ```solve_countdown``` starts the process and iterates over each number to start the calculation.

This approach is more efficient than brute force and we can support that by the time it took to execute the computation.

# References 

https://easychair.org/publications/open/2L76

--- 
[1] Lynch, J. and Yan Weng. “The Computational Complexity of Finding Arithmetic Expressions With and Without Parentheses.” https://www.semanticscholar.org/reader/21f455ca0bbf9a355611bc0593dd1cf8a8d32584

“Countdown” numbers game: the lowdown https://www.daitx.com/2016/05/01/countdown-math/

Reverse Polish Notation: https://mathworld.wolfram.com/ReversePolishNotation.html


